# Calculate LI
This notebook calcualtes the lifted index values for us to train models with. See the README to learn what the Lifted Index is. It reads in the raw data-por IGRA data. It then unpivots the data, converting columns back into pressure levels. This data can then be used by MetPy to calculate the Lifted Index using the actual physics algorithms. A CSV file is geratated for each input file.

**This notebook will take a long time to complete!**

Update the following parameters in the first cell to accomodate your installation:

- SILVER_GPH20S10K_PATH - The location to read the transformed CSV files
- SILVER_LI_PATH - Location to save the calculated Lifted Index values

In [2]:
import math
import os
from datetime import datetime
import numpy as np
from metpy.calc import parcel_profile, lifted_index
from metpy.units import units
import olieigra

BRONZE_DATA_POR_PATH = '/Users/olievortex/lakehouse/default/Files/bronze/igra2/data-por'
SILVER_LI_PATH = '/Users/olievortex/lakehouse/default/Files/silver/igra2/li'

In [3]:
# If you need to install olieigra, uncomment and execute this line. View the README in the project root for instructions
# on how to build or download this file.
#%pip install /lakehouse/default/Files/libs/olieigra-0.0.1-py3-none-any.whl

In [4]:
# Make sure the destination folder exists
os.makedirs(SILVER_LI_PATH, exist_ok=True)

In [5]:
# from datetime import datetime
class LiftedIndex(olieigra.Callbacks):
    """Calculate the lifted index from data-por igra data"""

    def __init__(self, dst_path: str, min_effective_date: datetime):
        super().__init__()
        self.dst_path = dst_path
        self.min_effective_date = min_effective_date
        self.filtered = 0
        self.errors = 0
        self.data = 0
        self.hout = ''
        self.filename = ''
        self.writer = None

    def start_file(self, filename: str) -> bool:
        """Decide if we want to process the file. If so, reset state and start writing to a
        temporary file."""

        # An IGRA2 file should end with -data.txt
        if not filename.endswith('-data.txt'):
            print(f'Skipping {filename}. Not sure what to do with it.')
            return False

        # Set the desired destination filename
        dst_filename = f'{self.dst_path}/{filename}'
        dst_filename = dst_filename.replace("-data.txt", "-data-li.csv")

        # Skip this file if it has already been processed
        if os.path.exists(dst_filename):
            print(f'Skipping {filename}. Destination file already exists.')
            return False

        # If we got here, we are going to process the file
        print(f'Processing {filename}.')

        # Write to a temp file
        self.filename = dst_filename.replace('-data-li.csv', '-data-li.partial.csv')
        self.writer = open(self.filename, 'w', encoding='UTF-8')

        # Reset the filtered record count
        self.filtered = 0
        self.errors = 0
        self.data = 0
        self.hout = ''

        # Write the header row
        self.writer.write('id,effective_date,hour,li\n')

        # Tell olieigra to continue processing
        return True

    def finish_file(self, headers: int, rows: int):
        """File processing is complete. Clean up and provide user feedback."""

        # Close the temporary file
        self.writer.close()

        # Rename the temporary file
        dst_renamed = self.filename.replace('.partial.csv', '.csv')
        os.rename(self.filename, dst_renamed)

        # Calculate the number of records written
        loaded = headers - self.filtered - self.errors - self.data

        # Provide feedback to the user
        print(f" Read {headers} headers, {rows} lines. Filtered {self.filtered}. " +
              f"Errors {self.errors}. Bad data {self.data}. Wrote {loaded} records.")

    def parse_header(self, header: olieigra.HeaderModel) -> bool:
        """Transform the header record and start writing a record"""

        # Combine the separate fields into a date
        effective_date = datetime(header.year, header.month, header.day)

        # Skip the record if it is too old
        if effective_date < self.min_effective_date:
            self.filtered += 1
            return False

        # We may not write the row, so save the header columns in a variable for now
        self.hout = f'{header.id},{effective_date:%Y-%m-%d},{header.hour}'

        # Tell olieigra to process the body associated with this header
        return True
    
    def parse_body(self, body: list[olieigra.BodyModel]):
        """Perform some analytics and finish writing a record"""
        
        # Initialize
        usable_all = 0
        surface_nan = 1
        pres = []
        temp = []
        dp = []

        # Iterate through each record in the body
        for item in body:
            # We don't care about non-pressure records
            if item.type[0] == '3':
                continue

            # We don't want records that contain a NaN value
            if math.isnan(item.dpdp) | math.isnan(item.temp):
                continue

            # This is a usable record
            usable_all += 1

            # Flag that we found a valid surface record
            if item.type == '21':
                surface_nan = 0

            pres.append(item.pres / 100.0)
            temp.append(item.temp / 10.0)
            dp.append((item.temp - item.dpdp) / 10.0)

        # Quality checks
        if surface_nan == 1 or len(pres) < 20:
            self.data += 1
            return
        
        # Calulate lifted index
        li = self.calculate_li(pres, temp, dp)
        
        # If LI is NaN, there was an exception in the calulation
        if math.isnan(li):
            self.errors += 1
            return
        
        # Write record
        if surface_nan == 0:
            self.writer.write(f'{self.hout},{li:.1f}\n')

    def calculate_li(self, pres: list[float], temp: list[float], dp: list[float]) -> float:
        """Calculate the lifted index using metpy"""

        # Conver the units
        pres = np.array(pres) * units.hPa
        temp = np.array(temp) * units.degC
        dp = np.array(dp) * units.degC

        # Calculate parameters
        try:
            # Calculate the parcel profile
            prof = parcel_profile(pres, temp[0], dp[0])

            # The calculation throws an exception if the data doesn't reach the EL
            li = lifted_index(pres, temp, prof)

            return float(li.magnitude[0])
        except:
            return math.nan


In [6]:
# Set up for processing
callbacks = LiftedIndex(SILVER_LI_PATH, datetime(2000, 1, 1))
reader = olieigra.Reader(callbacks=callbacks)
crawler = olieigra.Crawler(reader=reader)

# Crawl and process files
crawler.crawl(BRONZE_DATA_POR_PATH)


Processing USM00072251-data.txt.


/var/folders/7x/y_0m3dbn3pl7shgklj8r717m0000gn/T/ipykernel_3660/3984590618.py:149: UserWarning: Interpolation point out of data bounds encountered
  li = lifted_index(pres, temp, prof)


 Read 34081 headers, 4532303 lines. Filtered 16310. Errors 33. Bad data 1350. Wrote 16388 records.
Processing USM00072357-data.txt.
 Read 26409 headers, 4084416 lines. Filtered 8706. Errors 44. Bad data 1025. Wrote 16634 records.
Processing USM00072250-data.txt.
 Read 77417 headers, 5455633 lines. Filtered 59498. Errors 18. Bad data 1537. Wrote 16364 records.
Processing USM00072456-data.txt.
 Read 54326 headers, 5136424 lines. Filtered 36402. Errors 38. Bad data 945. Wrote 16941 records.
Processing USM00072451-data.txt.
 Read 65711 headers, 4849890 lines. Filtered 47852. Errors 23. Bad data 1106. Wrote 16730 records.
Processing USM00072649-data.txt.
 Read 26612 headers, 3659546 lines. Filtered 9224. Errors 25. Bad data 946. Wrote 16417 records.
Processing USM00072261-data.txt.
 Read 53682 headers, 5133602 lines. Filtered 35986. Errors 86. Bad data 1479. Wrote 16131 records.
Processing USM00074560-data.txt.
 Read 21214 headers, 3761759 lines. Filtered 3539. Errors 87. Bad data 793. Wrot

/opt/miniconda3/envs/igra/lib/python3.11/site-packages/metpy/calc/thermo.py:1313: RuntimeWarning: overflow encountered in exp
  return mpconsts.nounit.sat_pressure_0c * np.exp(
/opt/miniconda3/envs/igra/lib/python3.11/site-packages/metpy/calc/thermo.py:1458: RuntimeWarning: invalid value encountered in divide
  return molecular_weight_ratio * partial_press / (total_press - partial_press)


 Read 61708 headers, 4823579 lines. Filtered 44399. Errors 18. Bad data 832. Wrote 16459 records.
Processing USM00072249-data.txt.
 Read 49737 headers, 4446304 lines. Filtered 31519. Errors 30. Bad data 627. Wrote 17561 records.
Processing USM00072747-data.txt.
 Read 61276 headers, 4818934 lines. Filtered 44039. Errors 46. Bad data 660. Wrote 16531 records.
Processing USM00072662-data.txt.
 Read 69285 headers, 4949468 lines. Filtered 51686. Errors 18. Bad data 1208. Wrote 16373 records.
Processing USM00072558-data.txt.
 Read 83354 headers, 5105789 lines. Filtered 65841. Errors 39. Bad data 1343. Wrote 16131 records.
Processing USM00074455-data.txt.
 Read 23193 headers, 3857507 lines. Filtered 5560. Errors 57. Bad data 536. Wrote 17040 records.
Processing USM00072764-data.txt.
 Read 78952 headers, 5027576 lines. Filtered 61574. Errors 12. Bad data 1031. Wrote 16335 records.
Processing USM00072659-data.txt.
 Read 21198 headers, 3718703 lines. Filtered 3693. Errors 22. Bad data 845. Wrote